In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=5000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [4]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            if "science" in cat:
                return True
            if "physics" in cat:
                return True
            if "chemistry" in cat:
                return True
        return False
    
    def text_preprocess(text):
        return text.replace("===", "\n").replace("==", "\n")

    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = "title: " + df_science["title"] + "\n" + df_science["text"].apply(text_preprocess)
    return df_science.sample(len(df_science)//20)

In [5]:
files = glob.glob("../data/wikipedia/*.parquet")

In [6]:
import time

In [7]:
import pickle

In [8]:
import json

In [13]:
from datetime import datetime as dt
import os

In [ ]:
texts = []

In [14]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
# 依頼
You are a professor at a science university and are creating a test for your students.
Using the given text, create a question in which you select the most appropriate statement from the five options in the question text. Also, extract the evidence for your answer.
The output should be an array in json format, with "prompt" as the problem statement, "A," "B," "C," "D," and "E" as choices, "answer" as the answer choice (one of A through E), and "basis" as the rationale. Please make sure that the answer choices are not all the same, e.g., all five answers are A.
In addition, five questions must be created per text. This means that the total number of questions created will be 5.

# text
## text 1 
title: {series['title']}

{series['text'].split("==")[0]}


# attention
Please create one question per text. So the total number of problems created will be five.
"""
    return prompt

def f(series):
    if series["A"] != series["A"]:
        if type(series["choices"]) == dict:
            for key in ["A", "B", "C", "D", "E"]:
                series[key] = series["choices"][key]
        elif type(series["choices"] == list):
            for i, key in enumerate(["A", "B", "C", "D", "E"]):
                series[key] = series["choices"][i]
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

for file in files:
    if os.path.basename(file) in ["a.parquet", "all.parquet"]:
        print(f"pass: {file}")d
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            for text_json in texts_json:
                text_json["wiki_id"] = series["id"]
                text_json["original_text"] = series["text"]
                texts.append(text_json)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
        if i % 100 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

pass: ../data/wikipedia/a.parquet
pass: ../data/wikipedia/all.parquet


../data/wikipedia/c.parquet:   1%|█▌                                                                                                                                                                                                               | 1/129 [00:22<48:34, 22.77s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_5080/1800176891.py", line 46, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_5080/4006552839.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)


This model's maximum context length is 16385 tokens. However, you requested 20920 tokens (15920 in the messages, 5000 in the completion). Please reduce the length of the messages or completion.
[
  {
    "prompt": "What is configuration entropy?",
    "A": "The portion of a system's entropy related to the positions of its constituent particles",
    "B": "The portion of a system's entropy related to the velocity or momentum of its constituent particles",
    "C": "The portion of a system's entropy related to the energy of its constituent particles",
    "D": "The portion of a system's entropy related to the spin configurations of its constituent particles",
    "E": "The portion of a system's entropy related to the number of conformations of a molecule",
    "answer": "A",
    "basis": "The text states that configuration entropy is the portion of a system's entropy that is related to discrete representative positions of its constituent particles."
  },
  {
    "prompt": "How is configu

../data/wikipedia/c.parquet:   6%|████████████▉                                                                                                                                                                                                    | 8/129 [02:07<33:58, 16.84s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_5080/1800176891.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 50 column 106 (char 2156)
../data/wikipedia/c.parquet:   7%|██████████████▌                                                                                                          

Expecting ',' delimiter: line 50 column 106 (char 2156)
[
  {
    "prompt": "When did the TV series 'Counterpart' premiere?",
    "A": "December 10, 2016",
    "B": "December 10, 2017",
    "C": "December 10, 2018",
    "D": "December 10, 2019",
    "E": "December 10, 2020",
    "answer": "B",
    "basis": "The prompt states that the TV series 'Counterpart' premiered on December 10, 2017."
  },
  {
    "prompt": "Who is the main actor in the TV series 'Counterpart'?",
    "A": "J. K. Simmons",
    "B": "Olivia Williams",
    "C": "Harry Lloyd",
    "D": "Nazanin Boniadi",
    "E": "Sara Serraiocco",
    "answer": "A",
    "basis": "The prompt mentions that J. K. Simmons stars in the TV series 'Counterpart' as Howard Silk."
  },
  {
    "prompt": "What is the premise of the TV series 'Counterpart'?",
    "A": "A scientist creates parallel Earths in 1987.",
    "B": "A flu pandemic kills millions in the Prime world.",
    "C": "Counterparts are used as spies and sleeper agents.",
    "D"

...

../data/wikipedia/w.parquet:  77%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                               | 34/44 [09:28<02:53, 17.39s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_5080/1800176891.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 10 column 306 (char 1042)
../data/wikipedia/w.parquet:  80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Expecting ',' delimiter: line 10 column 306 (char 1042)
[
    {
        "prompt": "What is the Weierstrass transform?",
        "A": "The Weierstrass transform is a smoothed version of a function obtained by averaging the values of the function weighted with a Gaussian centered at x.",
        "B": "The Weierstrass transform is a method to approximate a given integrable function arbitrarily well with analytic functions.",
        "C": "The Weierstrass transform is a low-pass filter that reduces higher frequencies more than lower ones.",
        "D": "The Weierstrass transform is a linear and translation-invariant assignment of a new function to each function.",
        "E": "The Weierstrass transform is a convolution transform defined via the heat equation.",
        "answer": "A",
        "basis": "The Weierstrass transform is defined as a smoothed version of a function obtained by averaging the values of the function weighted with a Gaussian centered at x. This is mentioned in the te

../data/wikipedia/x.parquet:  67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                      | 2/3 [00:43<00:21, 21.47s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_5080/1800176891.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 33 column 53 (char 1075)
../data/wikipedia/x.parquet: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Expecting ',' delimiter: line 33 column 53 (char 1075)
[
    {
        "prompt": "What is the genre of the novel Xavras Wyżryn?",
        "A": "Alternate history",
        "B": "Science fiction",
        "C": "Military fiction",
        "D": "Historical fiction",
        "E": "Romantic fiction",
        "answer": "C",
        "basis": "The text states that Xavras Wyżryn is a member of the military science fiction genre."
    },
    {
        "prompt": "When was Xavras Wyżryn first published?",
        "A": "1997",
        "B": "2004",
        "C": "1981",
        "D": "1920",
        "E": "1935",
        "answer": "A",
        "basis": "The text mentions that Xavras Wyżryn was first published in 1997."
    },
    {
        "prompt": "Who is the main character of Xavras Wyżryn?",
        "A": "Ian Smith",
        "B": "Colonel Xavras Wyżryn",
        "C": "Jewriej",
        "D": "Sienkiewicz's Triple",
        "E": "Two Bodyguards",
        "answer": "B",
        "basis": "The text desc


../data/wikipedia/z.parquet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:17<00:00, 17.24s/it]


In [22]:
series['original_text']

'title: Nebula Winners Fourteen\nNebula Winners Fourteen is an anthology of award winning science fiction short works edited by Frederik Pohl. It was first published in hardcover by Harper & Row in August 1980. The first British edition was published in hardcover by W. H. Allen in April 1981. Paperback editions followed from Star in the U.K. in March 1982 and Bantam Books in the U.S. in July 1982. \nSummary\n The book collects pieces that won or were nominated for the Nebula Awards for novel, novella, novelette and short story for the year 1979 and a few other pieces related to the awards, together with a piece by 1979 Grand Master award winner L. Sprague de Camp and an introduction by the editor. Not all nominees for the various awards are included. \nContents\n *"Introduction: A Guide to the Perplexed" (Frederik Pohl) *"The Persistence of Vision" [Best Novella winner, 1979] (John Varley) *"Stone" [Best Short Story winner, 1979] (Edward Bryant) *"A Glow of Candles, a Unicorn\'s Eye" [

In [23]:
for i in range(10):
    series = df_texts.sample(1).iloc[0]
    print(f"{series['prompt']} \n text: \n {series['original_text'].split('==')[0]}")
    print("----------------")

What is the maximum sustainable yield (MSY) usually higher than? 
 text: 
 title: Maximum sustainable yield
In population ecology and economics, maximum sustainable yield (MSY) is theoretically, the largest yield (or catch) that can be taken from a species' stock over an indefinite period. Fundamental to the notion of sustainable harvest, the concept of MSY aims to maintain the population size at the point of maximum growth rate by harvesting the individuals that would normally be added to the population, allowing the population to continue to be productive indefinitely. Under the assumption of logistic growth, resource limitation does not constrain individuals' reproductive rates when populations are small, but because there are few individuals, the overall yield is small. At intermediate population densities, also represented by half the carrying capacity, individuals are able to breed to their maximum rate. At this point, called the maximum sustainable yield, there is a surplus of i

In [40]:
df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

In [20]:
pd.DataFrame(texts)["A"].isnull().sum()

69

In [41]:
df_science = get_df(files[0])
df_science[df_science["id"] == '29637793']

,id,title,text,categories


In [42]:
df_science

,id,title,text,categories
368626,1908395,Artificial brain,title: Artificial brain\nAn artificial brain (...,"[Computational neuroscience, Robotics, Emergin..."
34268,10410698,Abdul Amir al-Jamri,title: Abdul Amir al-Jamri\nSheikh Abdul Amir ...,"[1938 births, 2006 deaths, Deaths from kidney ..."
233077,1958222,Amyloid beta,title: Amyloid beta\nAmyloid beta (Aβ or Abeta...,"[Peptides, Molecular neuroscience, Alzheimer's..."
9870,3621668,A Woman of the Iron People,title: A Woman of the Iron People\nA Woman of ...,"[1991 American novels, 1991 science fiction no..."
139719,38366604,Albert Spaier,title: Albert Spaier\nAlbert Spaier (9 July 18...,"[1883 births, 1934 deaths, Writers from Iași, ..."
...,...,...,...,...
62685,4474244,Actuarial reserves,"title: Actuarial reserves\nIn insurance, an ac...","[Actuarial science, Capital requirement de:Dec..."
357456,4260564,Arrival II,title: Arrival II\nArrival ll (alternatively t...,"[1998 films, 1998 science fiction films, Ameri..."
391514,32894329,Astrobiophysics,title: Astrobiophysics\nAstrobiophysics is a f...,"[Astrophysics, Biophysics .]"
10000,2824171,A World of Difference (novel),title: A World of Difference (novel)\nA World ...,"[1990 American novels, Novels set during the C..."
